### 풀이
- 이항계수 수식은 다음과 같다
  - $\binom{n}{r} = \frac {n!}{r!(n-r)!}$
- 13977(이항 계수와 쿼리)에서도 살펴봤지만, 최대 $n$ 의 팩토리얼까지 미리 구해두는 방식으로 풀 수 있다.
- 그런데, `gnCr`은 사용할 수 없다. 역원을 사용하는 특성상 $p$ 가 소수여야 하기 때문. 문제의 $p$ 인 $m$ 은 소수가 아닐 수 있다.


- 2004(조합 0의 개수) 의 풀이를 참고해보자.
  - 0의 개수라 함은 팩토리얼을 구성하는 수 중에서, 10으로 나뉘어 떨어지는 수의 개수를 세는 것과 같다.
  - 이는 곧 10의 소인수인 2와 5의 개수를 구하고, 2의 배수이면서 5의 배수인 수의 개수, 즉, 둘 중에 더 작은 개수를 구하면 된다.
  - 팩토리얼을 구할 때, 통분을 하면 그 수가 없어지는 것과 같다. 그걸 고려한 개수 공식은 다음과 같다.\
  $\binom{n}{r}$ 에 대한 소수 $p$ 의 개수 $k$ 는 다음과 같다.
  - $k = {n \over p} - {r \over p} - {(n-r) \over p}$
    - $p$ 는 소수이다. 그리고 2와 5만 구하면 됐었다.
    - 이 때 소수 $p$ 에 대해 어떤 항도 정수가 아니지 않다는 증명: https://kin.naver.com/qna/detail.nhn?d1id=11&dirId=1113&docId=53607443

- 위의 공식을 확장해야 한다. 
  - 곱해진 모든 수를 다 세어야 하기 때문. 
  - 우선, 팩토리얼을 분해해보자.
    - $n! = n_1 \cdot n_2 \cdot n_3 \cdots n_i, (1 \leq i \leq n)$
  - 각 항을 네가지 경우로 나눠서 생각할 수 있다.
    1. 1인 경우(trivial)
    2. 소수인 경우
        - 위의 $k$ 를 그대로 사용하면 된다.
    3. 소수와 소수의 곱인 경우
        - 이것도 $k$ 안에서 모두 고려됨을 알 수 있다.
          - (증명) 몰루. 근데 직관적으로 맞긴함..
    4. 소수의 거듭제곱인 경우
        - 특수하게 처리해야 한다. 
        - 2로 예시를 든다면, $n!$ 에 대해 2의 배수를 세었다는 것은, $n!$ 에 2가 한 번 곱해진 수를 센 것과 같다.
        - 그런데, $4, 8, 16, 32, \cdots$ 와 같이 2의 거듭제곱 꼴로 되어있는 수들은 2가 여러 번 곱해진 수이다.
        - 이를 올바르게 세려면 각 거듭제곱꼴을 `지수`번 만큼 추가로 더해주어야 한다.
  - 4 를 다시 표현해보자. 
    - 2의 배수를 세기 이전의 $n!$ 속에 포함된 2의 거듭제곱 꼴들의 총 개수들은 다음과 같다.
      - ($2^1$ 배수의 개수) + ($2^2$ 배수의 개수) + $\cdots$ ($2^{log_2 n}$ 배수의 개수)
    - 그런데, ($2^1$ 의 배수의 개수)를 세고나면, ($2^2$ 의 배수의 개수)는 ($2^1$ 의 배수의 개수)로 감소(reduce)된다.
    - 그리고 나머지 거듭제곱 꼴들도 동일하게 영향 받는다. 
    - 마지막 항만 $2^{{log_2 n} - 1}$ 으로 감소한다.
    - 그렇다면, 동일한 패턴으로 모든 항이 전부 소모될 때 까지 반복할 수 있다.
  - 소수 $p$ 에 대해, $n!$ 에서 $p$ 의 모든 거듭제골 꼴을 포함하는 개수를 세는 점화식은 다음과 같다.
    - $k_p$ 는 $n!$ 에서 소수 $p$ 의 개수
    - $\displaystyle {n \over p} + {n \over p^2} + \cdots {n \over p^{log_p n}} = \sum_{i=1}^{log_p n} k_p$

- 이제 $n$ 까지의 소수를 구하면 된다. 에라토스테네스의 체를 사용하면 된다.
  - $n$ 까지의 소수를 모은 집합을 $E$ 라고 할 때 각 수가 더해진 개수는
  - $\displaystyle \sum_{p \in E} \sum_{i=1}^{log_p n} k_p$
  - 위에껀 사실 구할필욘 없다. 이항계수의 값을 출력해야 하니까.
  - $p \in E$ 에 대해서 각각 몇 개있는지만 세두면 된다.

In [ ]:
import math
def sieve(n):
  if n < 2: return set()
  n += 1
  P = [1] * (n//2)
  for i in range(3, math.isqrt(n) + 1, 2):
    if P[i//2]: P[i**2//2::i] = [0]*((n-i**2-1)//(2*i) + 1)

  return [2] + [2*i+1 for i in range(1, n//2) if P[i]]

def sol() :
  N, K, M = map(int, input().split())
  S = sieve(N)
  cnt = [0] * len(S)
  
  for i, p in enumerate(S) :
    j = p
    while j <= N :
      cnt[i] += ((N // j) - (K // j) - ((N-K) // j))
      j *= p

  ans = 1
  for i, c in enumerate(cnt) :
    for _ in range(c) :
      ans = (ans * S[i]) % M
      if ans == 0 : return print(0)
  print(ans)

sol()

- 왤케 많아